# Implementation of YIN Algorithm

https://github.com/NValsted/VForScienceProjects/blob/master/YIN_pitch_detection/pitch_detection.py

In [19]:
from scipy.io import wavfile
from scipy import signal
import numpy as np

In [20]:
def f(x):
    f_0 = 1
    envelope = lambda x: np.exp(-x)
    return np.sin(2*np.pi*f_0*x) * envelope(x)

def ACF( f, W, t, lag):
    return np.sum( f[t : t + W] * f[lag + t : lag + t + W] )

def DF(f , W, t, lag):
    return ACF(f, W, t, 0) + ACF(f, W, t + lag, 0) - (2 * ACF(f, W, t, lag))

def CMNDF(f, W, t , lag):
    if lag == 0:
        return 1
    return DF(f, W, t, lag) / np.sum([DF(f, W, t, j+1) for j in range(lag)]) * lag

In [21]:
def detect_pitch( f, W, t, sample_rate, bounds, thresh = 0.1):
    CMNDF_vals = [CMNDF(f, W, t, i) for i in range(*bounds)]
    sample = None
    for i, val in enumerate(CMNDF_vals):
        if val < thresh:
            sample = i + bounds[0]
            break
    if sample is None:
        sample = np.argmin(CMNDF_vals) + bounds[0]


    return sample_rate / sample


In [26]:
sample_rate, data = wavfile.read(f'audios\singer.wav')
data = data.astype(np.float64)
windows_size = int(5 / 2000 * 44100)
bounds = [ 20, 2000 ]

pitches = []

for i in range (data.shape[0] // (windows_size+3)):
    pitches.append(detect_pitch(data, windows_size, i * windows_size, sample_rate, bounds))

C:\Users\FRANCI~1\AppData\Local\Temp/ipykernel_7628/338288665.py:1: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, data = wavfile.read(f'audios\singer.wav')


KeyboardInterrupt: 